# CNN training
공개 데이터와 keras 라이브러리를 이용해 CNN 모델을 학습해 봅니다.

fer 데이터셋을 로컬 저장소로 다운로드 받습니다. shell script를 이용하기 때문에 윈도우 유저는 수동으로 진행해 주세요.
데이터 폴더의 구조는 아래와 같습니다.

    └── data
        └── fer2013
            ├── fer2013.csv 
            ├── ...
            └── ...

In [7]:
import subprocess
import os
from glob import glob

# if not 'windows OS', windows can't use shell script
# please, download & unzip manually
print ('OS name: %s'%os.name)
if os.name != 'nt':
    if not os.path.exists('./data'):
        os.mkdir('./data') # make directory
        
    subprocess.call('ls ./data', shell=True) # view file list

    if not os.path.exists('./data/fer2013.tar'):
        subprocess.call('wget https://www.dropbox.com/s/ojuk9bjm4r5bpnt/fer2013.tar -P ./data', 
                        shell=True) # download dataset
    if not os.path.exists('./data/fer2013/fer2013.csv'):
        subprocess.call('tar xvf ./data/fer2013.tar -C ./data', shell=True) # unzip
print (glob('./data/fer2013/*'))

OS name: posix
['./data/fer2013/fer2013.csv', './data/fer2013/fer2013.bib', './data/fer2013/README']


In [8]:
import pandas as pd
import numpy as np
import cv2

pd_data = pd.read_csv('./data/fer2013/fer2013.csv')
print (pd_data)

       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
5            2  55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...     Training
6            4  20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...     Training
7            3  77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...     Training
8            3  85 84 90 121 101 102 133 153 153 169 177 189 1...     Training
9            2  255 254 255 254 254 179 122 107 95 124 149 150...     Training
10           0  30 24 21 23 25 25 49 67 84 103 120 125 130 139...     Training
11           6  39 75 78 58 58 45 49 48 103 156 81 4

In [9]:
pd_train_data = pd_data[pd_data['Usage']=='Training']
pd_val_data = pd_data[pd_data['Usage']=='PublicTest']
pd_test_data = pd_data[pd_data['Usage']=='PrivateTest']
print (pd_train_data)

       emotion                                             pixels     Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training
5            2  55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...  Training
6            4  20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...  Training
7            3  77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...  Training
8            3  85 84 90 121 101 102 133 153 153 169 177 189 1...  Training
9            2  255 254 255 254 254 179 122 107 95 124 149 150...  Training
10           0  30 24 21 23 25 25 49 67 84 103 120 125 130 139...  Training
11           6  39 75 78 58 58 45 49 48 103 156 81 45 41 38 49...  Training
12          

In [10]:
label_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

data_usages = ['train', 'val', 'test']
dict_usages = {'train':pd_train_data, 'val':pd_val_data, 'test':pd_test_data}

for usage in data_usages:
    if not os.path.exists('./data/%s'%usage):
        os.mkdir('./data/%s'%usage)
        for idx, label_name in enumerate(label_names):
            os.mkdir('./data/%s/%02d.%s'%(usage, idx, label_name))

    np_data = dict_usages[usage].values
    np_images_flatten = np_data[:,1]
    np_label = np_data[:,0]

    print (np_label.shape)
    print (np_images_flatten.shape)

    for idx in range(np_label.shape[0]):
        if type(np_images_flatten[idx]) != str:
            print ('Image is not found, %d'%idx)
            continue
        img_flatten = np.array(np_images_flatten[idx].split(' ')).astype(np.uint8)
        label = np_label[idx]
        img_2d = np.reshape(img_flatten, (48,48))
        output_label_path = os.path.join('./data',usage,'%02d.%s'%(label, label_names[label]), 
                                         '%06d.png'%idx)
#         cv2.imwrite(output_label_path, img_2d)
        if idx%1000==0:
            print ('%s:(%d/%d), %s saved!'%(usage, idx, np_label.shape[0], output_label_path))

(28709,)
(28709,)
train:(0/28709), ./data/train/00.Angry/000000.png saved!
train:(1000/28709), ./data/train/06.Neutral/001000.png saved!
train:(2000/28709), ./data/train/06.Neutral/002000.png saved!
train:(3000/28709), ./data/train/02.Fear/003000.png saved!
train:(4000/28709), ./data/train/04.Sad/004000.png saved!
train:(5000/28709), ./data/train/03.Happy/005000.png saved!
train:(6000/28709), ./data/train/04.Sad/006000.png saved!
train:(7000/28709), ./data/train/04.Sad/007000.png saved!
train:(8000/28709), ./data/train/00.Angry/008000.png saved!
train:(9000/28709), ./data/train/06.Neutral/009000.png saved!
train:(10000/28709), ./data/train/03.Happy/010000.png saved!
train:(11000/28709), ./data/train/05.Surprise/011000.png saved!
train:(12000/28709), ./data/train/03.Happy/012000.png saved!
train:(13000/28709), ./data/train/04.Sad/013000.png saved!
train:(14000/28709), ./data/train/06.Neutral/014000.png saved!
train:(15000/28709), ./data/train/00.Angry/015000.png saved!
train:(16000/2870

In [18]:
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, Dropout
from keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D
from keras.layers import SeparableConv2D
# from keras.layers import Flatten
from keras import layers
from keras.regularizers import l2
from keras.models import Model

In [19]:
def XCEPTION(input_shape=None, l2_regularization=0.01):
    img_size = 48
    num_classes = 7

    # base
    if input_shape is not None:
        img_input = Input(input_shape)
    else:
        img_input = Input((img_size, img_size, 3))
    x = Conv2D(32, (3, 3), strides=(2, 2), use_bias=False)(img_input)
    x = BatchNormalization(name='block1_conv1_bn')(x)
    x = Activation('relu', name='block1_conv1_act')(x)
    x = Conv2D(64, (3, 3), use_bias=False)(x)
    x = BatchNormalization(name='block1_conv2_bn')(x)
    x = Activation('relu', name='block1_conv2_act')(x)

    residual = Conv2D(128, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name='block2_sepconv1_bn')(x)
    x = Activation('relu', name='block2_sepconv2_act')(x)
    x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name='block2_sepconv2_bn')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    residual = Conv2D(256, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = Activation('relu', name='block3_sepconv1_act')(x)
    x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name='block3_sepconv1_bn')(x)
    x = Activation('relu', name='block3_sepconv2_act')(x)
    x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name='block3_sepconv2_bn')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])
    x = Conv2D(num_classes, (3, 3),
            #kernel_regularizer=regularization,
            padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    output = Activation('softmax',name='predictions')(x)

    model = Model(img_input, output)
    return model

In [20]:
def centering(np_image):
    return 2*(np_image - 128)

In [21]:
from keras.optimizers import Adam

In [22]:
model = XCEPTION()
model.compile(optimizer=Adam(lr=0.001, decay=1e-6), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [23]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

import math
import cv2
import numpy as np
import os
from glob import glob
import argparse
from keras.utils import plot_model

In [24]:
img_size = 48
batch_size = 100

train_datagen = ImageDataGenerator(
            preprocessing_function=centering,
            rescale=1./255,
            shear_range=0.1,
            zoom_range=0.1,
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True
            )

test_datagen = ImageDataGenerator(
            preprocessing_function=centering,
            rescale=1./255
            )

In [25]:
train_generator = train_datagen.flow_from_directory(
            './data/train',
            target_size=(img_size, img_size),
            batch_size=batch_size,
            # color_mode='grayscale',
            class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
            './data/val',
            target_size=(img_size, img_size),
            batch_size=batch_size,
            shuffle=False,
            # color_mode='grayscale',
            class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [26]:
flgUsePretrainedWeight = True
if flgUsePretrainedWeight == True:
    model.load_weights('./models/xception/weights.20_0.61.h5')
    print ("[*] loaded pretrained model weight")

[*] loaded pretrained model weight


In [27]:
if not os.path.exists('./models/new_xception'):
    os.mkdir('./models/new_xception')
    
model_json = model.to_json()
with open('./models/new_xception/model.json', 'w') as json_file:
    json_file.write(model_json)
    
model.summary()
# plot_model(model, to_file='./models/new_xception/model.png')
# graph_model = cv2.imread('./models/new_xception/model.png', 1)
# import matplotlib.pyplot as plt
# plt.imshow(graph_model)
# plt.show()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 48, 48, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 23, 23, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 23, 23, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 23, 23, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
conv2d_2 (

In [28]:
model_checkpoint = ModelCheckpoint(
        './models/xception/weights.{epoch:02d}_{val_acc:0.2f}.h5', 
        save_best_only=False,
        verbose=1,
        monitor='val_loss',
        period= 2, #self.flag.total_epoch // 10 + 1, 
        save_weights_only=True)

def lr_step_decay(epoch):
        init_lr = 0.0001
        lr_decay = 0.5
        epoch_per_decay = 3
        lrate = init_lr * math.pow(lr_decay, math.floor((1+epoch)/epoch_per_decay))
        # print lrate
        return lrate

learning_rate = LearningRateScheduler(lr_step_decay)
callback_list = [model_checkpoint, learning_rate]

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=train_generator.n // batch_size,
        callbacks=callback_list
        )

Epoch 1/20
 30/287 [==>...........................] - ETA: 11:03 - loss: 1.0021 - acc: 0.6250